In [6]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.1.


# One classifier each author (OCEA)
1. This method throw away the authors write less than 100 papers
2. This method will convert muti-class case to mutiple OVR binary classifier.

In [1]:
import os
import sys
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import com_func

# parameters
#----- filter for selecting set of name group -----------#
filter_select_name_group = 100
#----- filter for selecting productive authors ----#
filter_lower = 100
filter_upper = 110

Dataset = "pubmed"

In [8]:
# text embedding only
pp_text_emb = ["tf", "tf_idf", "lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["off"]

In [9]:
# citation embedding only
pp_text_emb = ["off"]
pp_citation_emb = ["n2v","node2vec"]

In [2]:
# combined embedding
pp_text_emb = ["lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["n2v"]

In [3]:
print(pp_text_emb)
print(pp_citation_emb)

['lsa', 'pv_dm', 'pv_dbow']
['n2v']


In [6]:
# load the file
import io
import collections
import numpy as np
import pandas as pd

from statistics import mean 

from sklearn.preprocessing import normalize
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# fix random seed for reproducibility
np.random.seed(1)

fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

diff_embedding_result = collections.defaultdict(list)

# ----------------------- different text embedding ----------------------#
for text_emb in pp_text_emb:
    print("Load text embedding: ", text_emb)
    # read pretrained embeddings
    all_text_embedding = com_func.read_text_embedding(emb_type=text_emb, training_size="140k")
    all_text_emb_pid = [emb[0] for emb in all_text_embedding]
    all_text_embedding = [emb[1:] for emb in all_text_embedding]
    
    for citation_emb in pp_citation_emb:
        print("Load citation embedding: ", citation_emb)
        all_citation_embedding = com_func.read_citation_embedding_sorted(emb_type = citation_emb)
        all_citation_emb_pid = [emb[0] for emb in all_citation_embedding]
        all_citation_embedding = [emb[1:] for emb in all_citation_embedding]
        
        diff_threshold_result = collections.defaultdict(list)

        # -------------- different filter (step by 10) -----------------------#
        for step_filter in range(filter_lower, filter_upper, 10):
            # collect statistic to output
            statistic_detail = collections.defaultdict(list)
            
            # ------- select useful name group in all name group --------------------#
            for file in listfiles:
                # group name
                temp = file.split("_")
                name = temp[1]+"_"+temp[-1]
                print("For name: ",name)
                # read needed content in labeled file
                labeled_data = com_func.read_pid_aid(fileDir+file)
                #----------- select name group contain productive author------------------------------------#
                #----------- (contain pair of author write more than 100 papers) ---------------------------#
                # count number of paper each author write based on author ID
                authorCounter = collections.Counter(labeled_data["authorID"])
                # remove name group that do not contain pair of author write more than 100 papers
                for k in list(authorCounter):
                    if authorCounter[k] < filter_select_name_group:
                        del authorCounter[k]
                # if only have one class or no class pass the filter, not applicable
                if(len(authorCounter)==0) or (len(authorCounter)==1):
                    print(name, " pass")
                elif len(authorCounter)>2:
                    print("Muti-class case, convert to mutiple binary case.")
                    
                    
                    
                    
                    
                    
                    
                else:
                    temp_orginal_sample_size = len(labeled_data)
                    #--------select authors in name group are very productive (more than filter)---------#
                    print("Total sample size before apply filter: ",len(labeled_data))
                    # count number of paper each author write based on author ID
                    paperCounter = collections.Counter(labeled_data["authorID"])
                    print(paperCounter)
                    print("Total author before apply threshoid: ", len(paperCounter))
                    # collect per class statistic
                    for k in list(paperCounter):
                        if paperCounter[k] < step_filter:
                            del paperCounter[k]
                    temp =list(paperCounter.keys())
                    print(temp)
                    print("Total author after apply threshoid: ", len(temp))
                    # remove samples that are smaller than filter
                    labeled_data = labeled_data[labeled_data.authorID.isin(temp)]
                    print("Total sample size after apply filter: ",len(labeled_data))
                    #------------ extract paper representation -------------------------------------------#
                    # shuffle the data
                    labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
                    # extract true label and pid
                    label = labeled_data["authorID"]
                    pid = labeled_data["paperID"]
                    # list of different data field
                    part_collection = []
                    # select feature wanted to fit to clustering/classification algorithm
                    # data part, text information
                    data_part_text = com_func.extract_embedding(all_text_embedding, all_text_emb_pid, pid)
                    print("Text embedding shape: ", data_part_text.shape)
                    part_collection.append(data_part_text)
                    # data part, citation information
                    data_part_citation = com_func.extract_embedding(all_citation_embedding, all_citation_emb_pid, pid)
                    data_part_citation.fillna(0, inplace=True)
                    print("Citation embedding shape: ", data_part_citation.shape)
                    part_collection.append(data_part_citation)
                    # merge different part of data data together by concatenate it all together
                    # remove empty emb (when emb set off)
                    part_collection = [part for part in part_collection if len(part)!=0]
                    if len(part_collection)>1:
                        combinedata = np.concatenate(part_collection,axis=1)
                    elif len(part_collection)==1:
                        if isinstance(part_collection[0], pd.DataFrame):
                            combinedata = part_collection[0].values
                        else:
                            combinedata = part_collection[0]
                    else:
                        print("No data available")
                        break
                    print("Final feature (combined embedding) shape: ", combinedata.shape)
                    statistic_detail["Name group"].append(name)
                    statistic_detail["Class number"].append(len(paperCounter))
                    statistic_detail["Per class size"].append(paperCounter)
                    statistic_detail["Orginal sample size"].append(temp_orginal_sample_size)
                    statistic_detail["Total selected sample size"].append(len(labeled_data))
                    # -------------- using converted feature vector to train classifier-------------------#
                    if text_emb == "tf":
                        # using multinomial naive bayes
                        clf = MultinomialNB()    
                        mnbaccuracy, mnbmarcof1= com_func.k_fold_cv(combinedata, label, clf, k=10, verbose=True)
                        print("MNB F1: ", mnbmarcof1)
                        statistic_detail['MNB Accuracy'].append(mnbaccuracy)
                        statistic_detail['MNB macro F1'].append(mnbmarcof1)
                    # using logistic regression
                    clf = LogisticRegression(solver= "liblinear")
                    LRaccuracy, LRmarcof1 = com_func.k_fold_cv(combinedata, label, clf, k=10, verbose=True)
                    print("LR F1: ", LRmarcof1)
                    statistic_detail["LR accuracy"].append(LRaccuracy)
                    statistic_detail["LR macro f1"].append(LRmarcof1)
                    # using SVM with linear kernal
                    clf = SVC(gamma="auto", kernel='linear')
                    svcaccuracy, svcmarcof1 = com_func.k_fold_cv(combinedata, label, clf, k=10, verbose=True)
                    print("SVM F1: ", svcmarcof1)
                    statistic_detail["SVM(linear) accuracy"].append(svcaccuracy)
                    statistic_detail["SVM(linear) macro f1"].append(svcmarcof1)

            # write evaluation result to excel
            output = pd.DataFrame(statistic_detail)
            print(output)

            #savePath = "../../result/"+Dataset+"/3_OCEA_sample=140k/"
            #filename = "citation="+citation_emb+"_textual="+text_emb+"_threshold="+str(step_filter)+".csv"
            #com_func.write_csv_df(savePath, filename, output)
            print("Done")
            
            diff_threshold_result[step_filter].append(statistic_detail)
        
        diff_embedding_result["text="+text_emb+"citation="+citation_emb]
    

Load text embedding:  lsa
Total text vector records: 135796
Vector dimension:  100
Load citation embedding:  n2v
Total citation vector records: 124922
Vector dimension:  100
For name:  j_read
j_read  pass
For name:  f_esteves
f_esteves  pass
For name:  c_miller
c_miller  pass
For name:  r_jha
r_jha  pass
For name:  a_lowe
a_lowe  pass
For name:  a_vega
a_vega  pass
For name:  k_smith
k_smith  pass
For name:  j_gordon
j_gordon  pass
For name:  s_liao
s_liao  pass
For name:  j_qian
j_qian  pass
For name:  s_bernardi
s_bernardi  pass
For name:  t_hill
t_hill  pass
For name:  s_schindler
s_schindler  pass
For name:  j_williams
j_williams  pass
For name:  s_jacobson
s_jacobson  pass
For name:  e_andrade
e_andrade  pass
For name:  t_santos
t_santos  pass
For name:  k_kim
Muti-class case, convert to mutiple binary case.
For name:  d_ricci
d_ricci  pass
For name:  s_cameron
s_cameron  pass
For name:  t_wright
t_wright  pass
For name:  r_cunha
r_cunha  pass
For name:  s_fuchs
s_fuchs  pass
For 

Total missing sample:  12
Citation embedding shape:  (267, 100)
Final feature (combined embedding) shape:  (267, 200)
                     precision    recall  f1-score   support

0000-0002-3650-9381       1.00      0.99      1.00       154
0000-0003-1673-2954       0.99      1.00      1.00       113

           accuracy                           1.00       267
          macro avg       1.00      1.00      1.00       267
       weighted avg       1.00      1.00      1.00       267

[153   1   0 113]
LR F1:  0.9961686923330799
                     precision    recall  f1-score   support

0000-0002-3650-9381       1.00      0.98      0.99       154
0000-0003-1673-2954       0.97      1.00      0.99       113

           accuracy                           0.99       267
          macro avg       0.99      0.99      0.99       267
       weighted avg       0.99      0.99      0.99       267

[151   3   0 113]
SVM F1:  0.9885317488725034
For name:  a_biswas
a_biswas  pass
For name:  j_day
j

KeyboardInterrupt: 

In [ ]:
%reset

In [ ]:
%who